In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import importlib
import SmoothBrainChess
import pandas as pd
import numpy as np

In [5]:
importlib.reload( SmoothBrainChess )
input_file = '10k_INPUT_LAYERS.csv'
output_file = '10k_OUTPUT_NODE_INDICES.csv'

df_input = pd.read_csv(input_file)
df_output = pd.read_csv(output_file)

In [26]:
# CONVERT EMBEDDINGS FROM CSV
print(torch.cuda.is_available())
in_mat = df_input.to_numpy()
colors = in_mat[:,1]
embeddings = in_mat[:,2:].astype(np.float32)
torch_embeds = torch.from_numpy(embeddings).to('cuda')
print(torch_embeds.device)
print(torch_embeds.dtype)

True
cuda:0
torch.float32


In [27]:
outputs = df_output['OUT_INDEX'].to_numpy(dtype=int)
outputs

array([ 268, 1460,  317, ...,  482,  179,  502])

In [29]:
df_input[0:int(np.floor(len(df_input)/2))]

,INDEX,COLOR,IN_0,IN_1,IN_2,IN_3,IN_4,IN_5,IN_6,IN_7,...,IN_758,IN_759,IN_760,IN_761,IN_762,IN_763,IN_764,IN_765,IN_766,IN_767
0,0,WHITE,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,WHITE,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,WHITE,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3,WHITE,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,4,WHITE,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,WHITE,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4996,4996,WHITE,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4997,4997,WHITE,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4998,4998,WHITE,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [30]:
train=1
device='cuda'

df_input = pd.read_csv(input_file)
if train:
    df_input = df_input[0:int(np.floor(len(df_input)/2))]
else:
    df_input = df_input[int(np.ceil(len(df_input)/2)):]

# LOAD OUTPUTS
df_output = pd.read_csv(output_file)
if train:
    df_output = df_output[0:int(np.floor(len(df_output)/2))]
else:
    df_output = df_output[int(np.ceil(len(df_output)/2)):]

# EACH ELEMENT IN self.embeddings IS A NUMPY ARRAY OF THE INPUT LAYER EMBEDDING
embeddings = torch.from_numpy(df_input.to_numpy()[:,2:].astype(np.float32)).to(device)
# EACH ELEMENT IN self.labels IS THE INDEX OF THE CORRECT OUTPUT NODE
labels = torch.from_numpy(df_output['OUT_INDEX'].to_numpy(dtype=int)).to(device)

In [31]:
embeddings

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0')

In [32]:
labels

tensor([ 268, 1460,  317,  ..., 1600,  428,   21], device='cuda:0',
       dtype=torch.int32)

In [33]:
embeddings[0]

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [35]:
labels = 10
vec = torch.zeros(12,dtype=torch.float).scatter(0,torch.tensor(labels),value=1)
print(vec)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])


In [50]:
from functools import reduce
importlib.reload( SmoothBrainChess )
pth = 'training_layers_as_matrices\\'

"""train_data = SmoothBrainChess.TestChessDataSet_MultiFile(device, True,pth+'1.2M_INPUT_LAYERS_div',pth+'1.2M_OUTPUT_NODE_INDICES_div',2)
test_data = SmoothBrainChess.TestChessDataSet_MultiFile(device, False,pth+'1.2M_INPUT_LAYERS_div',pth+'1.2M_OUTPUT_NODE_INDICES_div',2)"""

device = 'cuda'
train = True
infile_pattern = pth+'1.2M_INPUT_LAYERS_div'
keyfile_pattern = pth+'1.2M_OUTPUT_NODE_INDICES_div'
ndiv = 2


print([infile_pattern+str(x+1)+'.csv' for x in range(ndiv)])
dfs_input = [pd.read_csv(infile_pattern+str(x+1)+'.csv') for x in range(ndiv)]
dfs_output = [pd.read_csv(keyfile_pattern+str(x+1)+'.csv') for x in range(ndiv)]

print('INPUT LENS: ', [x.shape[0] for x in dfs_input])
print('OUTPUT LENS: ', [x.shape[0] for x in dfs_output])

df_input = pd.concat(dfs_input)
df_output = pd.concat(dfs_output)

print('FINAL INPUT LEN: ',df_input.shape[0])
print('FINAL OUTPUT LEN: ',df_output.shape[0])

# HANDLE INPUTS
if train:
    df_input = df_input[0:int(np.floor(len(df_input)/2))]
else:
    df_input = df_input[int(np.ceil(len(df_input)/2)):]

# HANDLE OUPUTS OUTPUTS
if train:
    df_output = df_output[0:int(np.floor(len(df_output)/2))]
else:
    df_output = df_output[int(np.ceil(len(df_output)/2)):]

# EACH ELEMENT IN self.embeddings IS A NUMPY ARRAY OF THE INPUT LAYER EMBEDDING
embeddings = torch.from_numpy(df_input.to_numpy()[:,2:].astype(np.float32))

# EACH ELEMENT IN self.labels IS THE INDEX OF THE CORRECT OUTPUT NODE
label_indices = df_output['OUT_INDEX'].to_numpy()



['training_layers_as_matrices\\1.2M_INPUT_LAYERS_div1.csv', 'training_layers_as_matrices\\1.2M_INPUT_LAYERS_div2.csv']
INPUT LENS:  [30988, 30988]
OUTPUT LENS:  [30988, 30988]
FINAL INPUT LEN:  61976
FINAL OUTPUT LEN:  61976


In [49]:
print(dfs_input[0].shape)
print(dfs_input[1].shape)
dfs0 = pd.concat(dfs_input)
print(dfs0.shape)

(30988, 770)
(30988, 770)
(61976, 770)
